In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#https://scipython.com/blog/quadtrees-2-implementation-in-python/
#https://pydoc.net/openslide-python/1.1.1/openslide/

import numpy as np
import openslide
from probreg import cpd
from probreg import transformation as tf
import cv2
from PIL import Image
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json

import pickle

In [ ]:
## HACK get rid of it
import sys
sys.path.append("..")
from registration_tree import Rect, QuadTree

In [ ]:
paths = list(Path(r"D:/Datasets/ColonCancer/UKSH-HE/").glob("*.tif"))

if len(paths) == 0:
    paths = list(Path(r"/data/ScannerStudy/ColonCancer/UKSH-HE/").glob("*.tif"))

source_slide_paths = [path for path in paths if "IHC" in path.stem]
target_slide_paths = [path for path in paths if "IHC" not in path.stem]

In [ ]:
source_slide_dict = {name.stem.split("X40")[0]
                     .replace("AI", "A1").replace("1D", "D1")
                     .replace("1F", "F1").replace("1E", "E1")
                     .replace("1G", "G1").replace("1H", "H1") 
                     .replace(" H ", " H1 ").replace(" E ", " E1 ") 
                     .replace(" D ", " D1 ").replace(" F ", " F1 ").replace(" G ", " G1 ") 
                     .replace("CRC-A1-1 D1 ", "CRC-A1-1 ")
                     .replace("CRC-A1-4 D1 ", "CRC-A1-4 ")
                     .replace("CRC-A1-5 D1 ", "CRC-A1-5 ")
                     : 
                      {"name": name, "slide": openslide.OpenSlide(str(name))}
                      for name in source_slide_paths}
source_slide_dict.keys()

In [ ]:
target_slide_dict = {name.stem.split("40X")[0]
                     .replace("AI", "A1").replace("1D", "D1")
                     .replace("1F", "F1").replace("1E", "E1")
                     .replace("1G", "G1").replace("1H", "H1") : 
                      {"name": name, "slide": openslide.OpenSlide(str(name))}
                      for name in target_slide_paths}
target_slide_dict.keys()

In [ ]:
for name in source_slide_dict:
    if name not in target_slide_dict:
        print(name)

In [ ]:
for name in target_slide_dict:
    if name not in target_slide_dict:
        print(name)

In [ ]:
df = pd.read_csv("CCMCT.csv", delimiter=";", names=["image_name", "grade", "vector"])
df.head()

In [ ]:
df = pd.read_csv("CCMCT.csv", delimiter=";", names=["image_name", "grade", "vector"])
df["vector"] = [json.loads(vector)  for vector in df["vector"]]

df["x1"] = [vector['x1'] if type(vector) is dict else 0 for vector in df["vector"]]
df["y1"] = [vector['y1'] if type(vector) is dict else 0 for vector in df["vector"]]

df["x2"] = [vector['x2'] if type(vector) is dict else 0 for vector in df["vector"]]
df["y2"] = [vector['y2'] if type(vector) is dict else 0 for vector in df["vector"]]

df["center_x"] = [x1 + ((x2-x1) / 2) for x1, x2 in zip(df["x1"], df["x2"])]
df["center_y"] = [y1 + ((y2-y1) / 2) for y1, y2 in zip(df["y1"], df["y2"])]

df["anno_width"] = [x2-x1 for x1, x2 in zip(df["x1"], df["x2"])]
df["anno_height"]= [y2-y1 for y1, y2 in zip(df["y1"], df["y2"])]

df.head()

In [ ]:
%%time

results = {}
#target_depth = 2

for thumbnail_size in [(2048, 2048)]: #(1024, 1024), (2048, 2048), (4096, 4096)
    
    for target_depth in [0, 1, 2]:
        
        result_folder = Path(f"Depth_{target_depth}/{thumbnail_size[0]}")

        result_folder.mkdir(parents=True, exist_ok=True)
    
        for name, source in source_slide_paths.items():

            parameter = {
                # feature extractor parameters
                "point_extractor": "sift",  #orb , sift
                "maxFeatures": 512, 
                "crossCheck": False, 
                "flann": False,
                "ratio": 0.5, 
                "use_gray": False,

                # QTree parameter 
                "homography": True,
                "filter_outliner": False,
                "debug": True,
                "target_depth": target_depth,
                "run_async": False,
                "thumbnail_size": thumbnail_size
            }

            target = target_slide_paths[name]


            source_slide = source["slide"]
            target_slide = target["slide"]

            source_dimension = Rect.create(Rect, 0, 0, source_slide.dimensions[0], source_slide.dimensions[1])
            target_dimension = Rect.create(Rect, 0, 0, target_slide.dimensions[0], target_slide.dimensions[1])

            qtree = QuadTree(source_dimension, source_slide, target_dimension, target_slide, **parameter)

            results[source["name"]] = qtree
            
            with open(result_folder / f'{qtree.source_name}-To-{qtree.target_name}.pickle', 'wb') as handle:
                pickle.dump(qtree, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for path, qtree in results.items():
    
    if qtree.target_depth != 2:
        continue
    
    df_gt = df[df["image_name"] == path.name]#.sample(n=5, random_state=1)
    
    annos = np.array(df_gt[["center_x", "center_y", "anno_width", "anno_height"]]) 
    qtree.draw_annotations(annos, num_sub_pic=5, figsize=(10, 10))

In [ ]:
for path, qtree in results.items():
    
    if qtree.target_depth != 2:
        continue
        
    print(qtree)